# 14장 - 텐서플로의 구조 자세히 알아보기 (part 2)

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd

from IPython.display import Image

In [4]:
print(tf.__version__)
print(np.__version__)
print(pd.__version__)

2.5.0
1.19.5
1.4.3


# 텐서플로 추정기

이 절에서는 방향을 바꾸어 텐서플로 추정기(Estimator)를 사용한다.  
`tf.estimator` API는 훈련, 예측(추론), 평가와 같은 머신 러닝 작업의 기본 단계를 캡슐화한다. 추정기는 이 장에서 다룬 다른 방법에 비해 캡슐화가 잘되어 있을 뿐만 아니라 확장성도 높다.  
`tf.estimator` API는 주요 코드를 변경하지 않고 다양한 플랫폼에서 모델을 실행할 수 있도록 지원한다. 상용 애플리케이션의 소위 '제품화 단계'에 적합하다.  
또한, 텐서플로는 널리 사용하는 일부 머신 러닝과 딥러닝 구조를 미리 구현한 추정기를 제공한다. 특정 방법이 어떤 데이터셋이나 문제에 적용할 수 있는지 빠르게 확인하는 것과 같은 비교 연구에 유용하다.   

사전에 준비된 추정기 사용하는 단계  
- **단계 1**: 데이터 로딩을 위해 입력 함수 정의하기  
- **단계 2**: 추정기와 데이터 사이를 연결하기 위해 특성 열 정의하기  
- **단계 3**: 추정기 객체를 만들거나 케라스 모델을 추정기로 바꾸기  
- **단계 4**: 추정기 사용하기: train() evalueate() predict()

In [5]:
tf.random.set_seed(1)
np.random.seed(1)

## 특성 열 사용

머신 러닝과 딥러닝 애플리케이션에서 다양한 종류의 특성을 볼 수 있다. 연속 특성, 순서가 없는 범주형 특성, 순서가 있는 범주형 특성이 있다.  
4장에서 다양한 종류의 특성을 다루는 방법을 배웠다. 수치형 데이터는 연속적이거나 이산적일 수 있지만 텐서플로 API에서는 '수치형' 데이터는 특별히 부동 소수 형태의 연속적인 데이터를 나타낸다.  

이따금 특성 집합은 다양한 특성 종류가 혼합되어 구성된다. 텐서플로 추정기는 이런 모든 종류의 특성을 다루도록 설계되었지만 각 특성이 추정기에서 어떻게 해석되어야 하는지 지정해 주어야 한다.

In [6]:
Image(url='https://git.io/JL56E', width=700)

위 그림에 있는 특성(model year, cylinders, displacement, horsepower, weigth, acceleration, origin)은 Auto MPG 데이터셋에서 가져온 것이다.  
이 데이터셋은 널리 사용되는 머신 러닝 벤치마크 데이터셋으로 갤런당 마일(MP) 단위로 자동차의 연료 효율성을 예측하는 문제이다.  

Auto MPG 데이터셋 중 다섯 개의 특성(cylinders, displacement, horsepower, weight, accleration)은 수치형(여기서는 연속적인)특성이다.  
model year는 순서가 있는 범주형 특성으로 볼 수 있다. 마지막으로 origin은 순서가 없는 특성이며 각각 US, 유럽, 일본을 의미하는 1, 2, 3 세 개의 값이 가능하다.  

먼저 데이터를 로드한 후 훈련 데이터셋과 테스트 데이터셋으로 나누고 연속적인 특성에 표준화를 적용하는 것과 같은 필수적인 전처리 단계를 수행해보자.

In [7]:
dataset_path = tf.keras.utils.get_file("auto-mpg.data", 
                                       ("http://archive.ics.uci.edu/ml/machine-learning-databases"
                                        "/auto-mpg/auto-mpg.data"))

column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower',
                'Weight', 'Acceleration', 'ModelYear', 'Origin']

df = pd.read_csv(dataset_path, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)

df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [8]:
## NA 열 삭제
print(df.isna().sum())

df = df.dropna()
df = df.reset_index(drop=True)
df.tail()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
ModelYear       0
Origin          0
dtype: int64


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1
391,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [9]:
## 훈련/테스트 분할
import sklearn
import sklearn.model_selection


df_train, df_test = sklearn.model_selection.train_test_split(df, train_size=0.8)
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.404153,7.666909,9.0,17.5,23.0,29.0,46.6
Cylinders,313.0,5.402556,1.701506,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,189.512780,102.675646,68.0,104.0,140.0,260.0,455.0
Horsepower,313.0,102.929712,37.919046,46.0,75.0,92.0,120.0,230.0
Weight,313.0,2961.198083,848.602146,1613.0,2219.0,2755.0,3574.0,5140.0
Acceleration,313.0,15.704473,2.725399,8.5,14.0,15.5,17.3,24.8
ModelYear,313.0,75.929712,3.675305,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.591054,0.807923,1.0,1.0,1.0,2.0,3.0


In [10]:
numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']

df_train_norm, df_test_norm = df_train.copy(), df_test.copy()

for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std  = train_stats.loc[col_name, 'std']
    df_train_norm.loc[:, col_name] = (df_train_norm.loc[:, col_name] - mean)/std
    df_test_norm.loc[:, col_name] = (df_test_norm.loc[:, col_name] - mean)/std
    
df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
203,28.0,-0.824303,-0.901020,-0.736562,-0.950031,0.255202,76,3
255,19.4,0.351127,0.413800,-0.340982,0.293190,0.548737,78,1
72,13.0,1.526556,1.144256,0.713897,1.339617,-0.625403,72,1
235,30.5,-0.824303,-0.891280,-1.053025,-1.072585,0.475353,77,1
37,14.0,1.526556,1.563051,1.636916,1.470420,-1.359240,71,1


앞의 코드에서 만든 판다스의 `DataFrame`은 `float` 타입의 열이 다섯 개 있다. 이 열들은 연속적인 특성으로 구성될 것이다.  
다음 코드에서 텐서플로의 `feature_column` 함수를 사용하여 이 연속적인 특성을 텐서플로의 추정기가 다룰 수 있는 특성 열 데이터 구조로 변환한다. 

In [11]:
numeric_features = []

for col_name in numeric_column_names:
    numeric_features.append(tf.feature_column.numeric_column(key=col_name))
    
numeric_features

[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

그다음 훈련 모델의 학습 작업을 단순화하기 위해 `model year` 정보를 그룹으로 묶어 버킷(bucket)으로 나눈다.  
구체적으로 말해서 다음과 같이 각 자동차를 네 개 중 한개의 'year' 버킷에 할당한다.  
$$버킷 = \begin{cases}0 & year < 73 일\, 때 \\  
                     1 & 73 \le year \le 76 일\, 때\\
                     2 & 76 \le year \le 79 일\, 때\\
                     3 & year \ge 79 일\, 때
                     \end{cases}$$  

이 간격은 버킷 개념을 설명하기 위해 임의로 선택한 것이다. 자동차를 세 개의 버킷으로 그룹짓기 위해 먼저 원본 `model year`를 기반으로 수치 특성을 정의한다. 그다음 이 수치 특성을 `[73, 76, 79]` 세 개의 분할 기준을 정한 bucketized_column 함수에 전달한다. 이 값은 분할의 오른쪽 기준이다.

In [12]:
feature_year = tf.feature_column.numeric_column(key="ModelYear")

bucketized_features = []

bucketized_features.append(tf.feature_column.bucketized_column(
    source_column=feature_year,
    boundaries=[73, 76, 79]))

print(bucketized_features)

[BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79))]


버킷으로 나눈 이 특성 열이 하나이지만 수치 특성과 마찬가지로 동일하게 파이썬 리스트에 추가한다. 다음 단계에서 이 리스트를 다른 특성으로 만든 리스트와 합친다. 이를 텐서플로의 추정기 기반 모델에 입력으로 제공한다.  

그 다음 순서가 없는 범주 특성 `Origin`을 위해 리스트를 정의한다. 텐서플로에는 범주 특성 열을 만드는 방법이 여러 가지 있다. 데이터가 범주 이름(예를 들어 "US", "Europe", "Japan"과 같이 문자열 포맷)이라면 `tf.feature_column.categorical_column_with_vocabulary_list`를 사용하여 가능한 고유 범주 이름을 입력으로 제공할 수 있다.  
전형적인 텍스트 분석의 경우처럼 가능한 범주 리스트가 너무 크다면 대신 `tf.feature_column.categorical_column_with_vocabulary_file`을 사용할 수 있다. 이 함수를 사용할 때 모든 범주/단어가 들어 있는 파일을 제공하므로 메모리에 모든 가능한 단어 목록을 유지할 필요가 없다. 또한, 특성이 [0, num_categories) 범위의 범주 인덱스에 이미 매핑되어 있다면 `tf.feature_column.categorical_column_with_identity` 함수를 사용할 수 있다. 하지만 여기서는 특성 Origin이 정수 1, 2, 3으로 주어지기 때문에 범주 인덱스가 맞지 않다. 인덱스는 0부터 시작해야 하기 때문이다.

In [13]:
feature_origin = tf.feature_column.categorical_column_with_vocabulary_list(
    key='Origin',
    vocabulary_list=[1, 2, 3])

DNNClassifier와 DNNRegressor 같은 추정기는 소위 밀집 열(dense column)만 받는다. 따라서 다음 단계는 이 범주 특성 열을 밀집 열로 바꾸는 것이다.  
두 가지 방법이 있다. `embedding_column` 함수를 사용하여 임베딩 열로 바꾸거나 `indicator_column` 함수로 인디케이터 열로 바꾸는 것이다.  
예를 들어 인덱스 0은 [1, 0, 0]으로 인코딩 되고, 인덱스 1은 [0, 1, 0]으로 인코딩되는 식이다.  
반면 임베딩 열을 각 인덱스를 float 타입의 랜덤한 수치 벡터로 매핑한다.  

범주 개수가 많을 때는 범주 개수보다 적은 차원의 임베딩 열을 사용하여 성능을 높일 수 있다.

In [14]:
categorical_indicator_features = []
categorical_indicator_features.append(tf.feature_column.indicator_column(feature_origin))

print(categorical_indicator_features)

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0))]


## 사전에 준비된 추정기로 머신 러닝 수행

필수적인 특성 열을 구성한 후 텐서플로 추정기를 사용할 수 있다. 사전 준비된 추정기를 사용하는 단계는 네 가지로 요약할 수 있다.  
1. 데이터 로드를 위해 입력 함수를 정의한다.  
2. 데이터셋을 특성 열로 변환한다.  
3. 추정기 객체를 만든다(사전 준비된 추정기를 사용하거나 케라스 모델을 추정기로 변환하여 새로 만든다).
4. 추정기의 train(), evaluate(), predict() 메서드를 사용한다.  

In [15]:
def train_input_fn(df_train, batch_size=8):
    df = df_train.copy()
    train_x, train_y = df, df.pop('MPG')
    dataset = tf.data.Dataset.from_tensor_slices((dict(train_x), train_y))

    # 셔플, 반복, 배치
    return dataset.shuffle(1000).repeat().batch(batch_size)

## 조사
ds = train_input_fn(df_train_norm)
batch = next(iter(ds))
print('키:', batch[0].keys())
print('ModelYear:', batch[0]['ModelYear'])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

키: dict_keys(['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'ModelYear', 'Origin'])
ModelYear: tf.Tensor([82 78 76 72 78 73 70 78], shape=(8,), dtype=int64)


2022-11-23 12:18:48.240911: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-23 12:18:48.241028: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


이 함수에서 `dict(train_x)`로 판다스 DataFrame 객체를 파이썬 딕셔너리로 바꾸었다.  

모델을 훈련하고 나서 평가할 때 사용할 테스트 데이터셋을 위해서도 입력 함수를 정의해야 한다.

In [16]:
def eval_input_fn(df_test, batch_size=8):
    df = df_test.copy()
    test_x, test_y = df, df.pop('MPG')
    dataset = tf.data.Dataset.from_tensor_slices((dict(test_x), test_y))

    return dataset.batch(batch_size)

두 번째 단계로 넘어가서 특성 열을 정의한다. 연속적인 특성을 담은 리스트, 버킷으로 나눈 특성 열을 위한 리스트, 범주 특성 열을 위한 리스트를 이미 만들어 놓았다.  
이런 개별 리스트를 연결하여 모든 특성 열을 담은 리스트를 만든다.

In [17]:
all_feature_columns = (numeric_features + 
                       bucketized_features + 
                       categorical_indicator_features)

print(all_feature_columns)

[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0))]


세 번째 단계에서 추정기 객체를 만든다. MPG 값을 예측하는 것은 전형적인 회귀 문제이므로 `tf.estimator.DNNRegressor`를 사용한다.  
회귀 추정기를 만들 때 특성 열의 리스트와 은닉층의 은닉 유닛 개수를 `hidden_units` 매게변수에 지정해야 한다. 여기서는 두 개의 은닉층을 사용한다.

In [18]:
regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10], # 첫 번째 은닉층은 32개의 유닛, 두 번째 은닉층은 10개의 유닛을 가진다.
    model_dir='models/autompg-dnnregressor/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/autompg-dnnregressor/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


또 `model_dir` 매개변수에 모델 파라미터가 저장된 디렉터리를 지정한다. 추정기의 장점 중 하나는 훈련하는 동안 자동으로 모델의 체크포인트 파일을 저장하는 것이다. 따라서 예기치 않은 이유로 모델 훈련이 중지되었을 때 마지막으로 저장된 체크포인트를 로드하여 훈련을 이어갈 수 있다.  
체크포인트 파일은 `model_dir` 매개변수로 지정된 디렉터리에 저장된다. `model_dir` 매개변수를 지정하지 않았다면 추정기가 랜덤한 임시 폴더를 만들어 체크포인트를 저장한다.  

세 개의 기본 단계를 끝내야 비로소 추정기를 사용하여 훈련, 평가, 예측을 수행할 수 있다. 

In [19]:
EPOCHS = 1000
BATCH_SIZE = 8
total_steps = EPOCHS * int(np.ceil(len(df_train) / BATCH_SIZE))
print('훈련 스텝:', total_steps)

regressor.train(
    input_fn=lambda:train_input_fn(df_train_norm, batch_size=BATCH_SIZE),
    steps=total_steps)

훈련 스텝: 40000
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2022-11-23 12:18:48.921692: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-23 12:18:48.921708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-23 12:18:48.929558: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-23 12:18:48.929823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:18:48.941386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:18:48.948331: I tensorflow/core/grappler/o

INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into models/autompg-dnnregressor/model.ckpt.


2022-11-23 12:18:49.425779: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 660.3955, step = 0


2022-11-23 12:18:49.825772: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:18:49.833365: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:18:50.017302: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:global_step/sec: 80.7978
INFO:tensorflow:loss = 513.2289, step = 100 (1.238 sec)
INFO:tensorflow:global_step/sec: 87.6665
INFO:tensorflow:loss = 536.07684, step = 200 (1.140 sec)
INFO:tensorflow:global_step/sec: 106.385
INFO:tensorflow:loss = 593.9197, step = 300 (0.940 sec)
INFO:tensorflow:global_step/sec: 107.279
INFO:tensorflow:loss = 605.18805, step = 400 (0.932 sec)
INFO:tensorflow:global_step/sec: 106.899
INFO:tensorflow:loss = 875.4829, step = 500 (0.935 sec)
INFO:tensorflow:global_step/sec: 105.221
INFO:tensorflow:loss = 742.5132, step = 600 (0.950 sec)
INFO:tensorflow:global_step/sec: 101.35
INFO:tensorflow:loss = 460.9865, step = 700 (0.987 sec)
INFO:tensorflow:global_step/sec: 101.447
INFO:tensorflow:loss = 398.84558, step = 800 (0.986 sec)
INFO:tensorflow:global_step/sec: 105.762
INFO:tensorflow:loss = 545.3564, step = 900 (0.946 sec)
INFO:tensorflow:global_step/sec: 101.499
INFO:tensorflow:loss = 680.369, step = 1000 (0.985 sec)
INFO:tensorflow:global_step/

`.train()`을 호출하면 자동으로 모델 훈련하는 과정 동안 체크포인트를 저장한다. 나중에 마지막 체크포인트를 다시 불러올 수 있다.

In [20]:
reloaded_regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10],
    warm_start_from='models/autompg-dnnregressor/',
    model_dir='models/autompg-dnnregressor/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/autompg-dnnregressor/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


그다음 훈련된 모델의 예측 성능을 평가하기 위해 다음과 같이 `evalueate()` 메서드를 사용할 수 있다. 

In [21]:
eval_results = reloaded_regressor.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

for key in eval_results:
    print('{:15s} {}'.format(key, eval_results[key]))
    
print('평균 손실 {:.4f}'.format(eval_results['average_loss']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-11-23T12:25:03
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2022-11-23 12:25:03.508771: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-23 12:25:03.508790: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-23 12:25:03.519199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:03.526443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:03.532879: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:03.54

INFO:tensorflow:Inference Time : 0.27546s
INFO:tensorflow:Finished evaluation at 2022-11-23-12:25:03
INFO:tensorflow:Saving dict for global step 40000: average_loss = 16.535028, global_step = 40000, label/mean = 23.611393, loss = 16.465263, prediction/mean = 21.969591


2022-11-23 12:25:03.736503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 40000: models/autompg-dnnregressor/model.ckpt-40000
average_loss    16.5350284576416
label/mean      23.611392974853516
loss            16.46526336669922
prediction/mean 21.96959114074707
global_step     40000
평균 손실 16.5350


In [22]:
pred_res = regressor.predict(input_fn=lambda: eval_input_fn(df_test_norm, batch_size=8))

print(next(iter(pred_res)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([23.059402], dtype=float32)}


2022-11-23 12:25:04.274371: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-23 12:25:04.274387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-23 12:25:04.280660: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:04.286104: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:04.292308: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:04.29

### Boosted Tree Regressor

In [23]:
boosted_tree = tf.estimator.BoostedTreesRegressor(
    feature_columns=all_feature_columns,
    n_batches_per_layer=20,
    n_trees=200)

boosted_tree.train(
    input_fn=lambda:train_input_fn(df_train_norm, batch_size=BATCH_SIZE))

eval_results = boosted_tree.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

print(eval_results)

print('평균 손실 {:.4f}'.format(eval_results['average_loss']))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/tb/x7hcv5j94gn9b46c7qc1vfth0000gn/T/tmp54lx3zxg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
The old _F

Exception ignored in: <function CapturableResource.__del__ at 0x13010b670>
Traceback (most recent call last):
  File "/Users/hanhyeongu/miniforge3/envs/hg/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 269, in __del__
    with self._destruction_context():
AttributeError: 'TreeEnsemble' object has no attribute '_destruction_context'


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2022-11-23 12:25:04.887430: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-23 12:25:04.887450: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-23 12:25:04.899680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:04.906847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:04.915187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:25:04.92

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/tb/x7hcv5j94gn9b46c7qc1vfth0000gn/T/tmp54lx3zxg/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'


...

INFO:tensorflow:loss = 0.00050202495, step = 23980 (0.949 sec)
INFO:tensorflow:global_step/sec: 103.767
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 24000...
INFO:tensorflow:Saving checkpoints for 24000 into /var/folders/tb/x7hcv5j94gn9b46c7qc1vfth0000gn/T/tmp54lx3zxg/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 24000...
INFO:tensorflow:Loss for final step: 0.00035642966.
INFO:tensorflow:Calling model_fn.


Exception ignored in: <function CapturableResource.__del__ at 0x13010b670>
Traceback (most recent call last):
  File "/Users/hanhyeongu/miniforge3/envs/hg/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py", line 269, in __del__
    with self._destruction_context():
AttributeError: 'TreeEnsemble' object has no attribute '_destruction_context'


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-11-23T12:28:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/tb/x7hcv5j94gn9b46c7qc1vfth0000gn/T/tmp54lx3zxg/model.ckpt-24000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2022-11-23 12:28:43.752840: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-23 12:28:43.752862: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-23 12:28:43.759749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:28:43.767725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:28:43.773923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-11-23 12:28:43.78

INFO:tensorflow:Inference Time : 0.24059s
INFO:tensorflow:Finished evaluation at 2022-11-23-12:28:43
INFO:tensorflow:Saving dict for global step 24000: average_loss = 10.688785, global_step = 24000, label/mean = 23.611393, loss = 10.607884, prediction/mean = 22.829304
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 24000: /var/folders/tb/x7hcv5j94gn9b46c7qc1vfth0000gn/T/tmp54lx3zxg/model.ckpt-24000
{'average_loss': 10.688785, 'label/mean': 23.611393, 'loss': 10.607884, 'prediction/mean': 22.829304, 'global_step': 24000}
평균 손실 10.6888
